In [1]:
import taxcalc as tc
import altair as alt
import pandas as pd
import numpy as np
from pathlib import Path
from collections import defaultdict

In [2]:
puf = pd.read_csv("puf_data/puf.csv", index_col=None)
cps = pd.read_csv("cps_data/pycps/cps.csv.gz", index_col=None)
cps_weights = pd.read_csv("cps_stage2/cps_weights.csv.gz", index_col=None)

In [3]:
START_YR = 2019
END_YR = 2029
epsilon = 1e-9

I hid some functions to make the notebook easier to read

In [4]:
def add_bins(dframe, income_measure, num_bins, wt='s006',
             decile_details=False,
             weight_by_income_measure=False,):
    """
    Add a variable to specified Pandas DataFrame, dframe, that specifies the
    table row and is called 'table_row'.  The rows hold equal number of
    filing units when weight_by_income_measure=False or equal number of
    income dollars when weight_by_income_measure=True.  Assumes that
    specified dframe contains columns for the specified income_measure and
    for sample weights, s006.  When num_quantiles is 10 and decile_details
    is True, the bottom decile is broken up into three subgroups (neg, zero,
    and pos income_measure ) and the top decile is broken into three subgroups
    (90-95, 95-99, and top 1%).
    """
    assert isinstance(dframe, pd.DataFrame)
    assert income_measure in dframe
    if decile_details and num_bins != 10:
        msg = 'decile_details is True when num_quantiles is {}'
        raise ValueError(msg.format(num_bins))
    dframe.sort_values(by=income_measure, inplace=True)
    if weight_by_income_measure:
        dframe['cumsum_temp'] = np.cumsum(
            np.multiply(dframe[income_measure].values, dframe[wt].values)
        )
        min_cumsum = dframe['cumsum_temp'].values[0]
    else:
        dframe['cumsum_temp'] = np.cumsum(dframe[wt].values)
        min_cumsum = 0.  # because s006 values are non-negative
    max_cumsum = dframe['cumsum_temp'].values[-1]
    cumsum_range = max_cumsum - min_cumsum
    bin_width = cumsum_range / float(num_bins)
    bin_edges = list(min_cumsum +
                     np.arange(0, (num_bins + 1)) * bin_width)
    bin_edges[-1] = 9e99  # raise top of last bin to include all observations
    bin_edges[0] = -9e99  # lower bottom of 1st bin to include all observations
    if decile_details:
        assert bin_edges[1] > 1e-9  # bin_edges[1] is top of bottom decile
        bin_edges.insert(1, 1e-9)  # top of zeros
        bin_edges.insert(1, -1e-9)  # top of negatives
        bin_edges.insert(-1, bin_edges[-2] + 0.5 * bin_width)  # top of 90-95
        bin_edges.insert(-1, bin_edges[-2] + 0.4 * bin_width)  # top of 95-99
        num_bins += 4
    labels = range(1, (num_bins + 1))
    dframe['bins'] = pd.cut(dframe['cumsum_temp'], bin_edges,
                            right=False, labels=labels)
    dframe.drop('cumsum_temp', axis=1, inplace=True)
    return dframe

In [5]:
def weighted_mean(pdf, col_name, wt_name='s006'):
    """
    Return weighted mean of col_name

    Parameters
    ----------
    pdf: Pandas DataFrame object
    col_name: variable to be averaged
    wt_name: weight
    """
    return (float((pdf[col_name] * pdf[wt_name]).sum()) /
            float(pdf[wt_name].sum() + epsilon))


def weighted_sum(pdf, col_name, wt_name='s006'):
    """
    Return weighted sum of col_name

    Parameters
    ----------
    pdf: Pandas DataFrame object
    col_name: variable to be averaged
    wt_name: weight
    """
    return (float((pdf[col_name] * pdf[wt_name]).sum()))

In [6]:
def percentile(pdf, col_name, num_bins, income_measure,
               wt='s006', income_wt=False,
               result_type='avg',
               decile_details=False):
    """
    """
    qpdf = add_bins(pdf, income_measure=income_measure, num_bins=num_bins,
                    wt=wt, decile_details=decile_details,
                    weight_by_income_measure=income_wt)
    gpdf = qpdf.groupby('bins', as_index=False)
    if result_type == 'avg':
        wpdf = gpdf.apply(weighted_mean, col_name)
    elif result_type == 'sum':
        wpdf = gpdf.apply(weighted_sum, col_name)
    else:
        msg = 'result_type must be "avg" or "sum"'
        raise ValueError(msg)
    return wpdf

In [7]:
def distribution(item, weight, agi):
    """
    Return distribution of item by AGI level
    """
    total = (item * weight).sum()
    agi_1 = ((item[agi < 0] * weight[agi < 0]).sum())
    pct1 = round(agi_1 / total, 2)
    agi_2 = ((item[(agi > 1) & (agi < 5000)] *
              weight[(agi > 1) & (agi < 5000)]).sum())
    pct2 = round(agi_1 / total, 2)
    agi_3 = ((item[(agi > 5000) & (agi < 10000)] *
              weight[(agi > 5000) & (agi < 10000)]).sum())
    pct3 = round(agi_3 / total, 2)
    agi_4 = ((item[(agi > 10000) & (agi < 15000)] *
              weight[(agi > 10000) & (agi < 15000)]).sum())
    pct4 = round(agi_4 / total, 2)
    agi_5 = ((item[(agi > 15000) & (agi < 20000)] *
              weight[(agi > 15000) & (agi < 20000)]).sum())
    pct5 = round(agi_5 / total, 2)
    agi_6 = ((item[(agi > 20000) & (agi < 25000)] *
              weight[(agi > 20000) & (agi < 25000)]).sum())
    pct6 = round(agi_6 / total, 2)
    agi_7 = ((item[(agi > 25000) & (agi < 30000)] *
              weight[(agi > 25000) & (agi < 30000)]).sum())
    pct7 = round(agi_7 / total, 2)
    agi_8 = ((item[(agi > 30000) & (agi < 40000)] *
              weight[(agi > 30000) & (agi < 40000)]).sum())
    pct8 = round(agi_8 / total, 2)
    agi_9 = ((item[(agi > 40000) & (agi < 50000)] *
              weight[(agi > 40000) & (agi < 50000)]).sum())
    pct9 = round(agi_9 / total, 2)
    agi_10 = ((item[(agi > 50000) & (agi < 75000)] *
               weight[(agi > 50000) & (agi < 75000)]).sum())
    pct10 = round(agi_10 / total, 2)
    agi_11 = ((item[(agi > 75000) & (agi < 100000)] *
               weight[(agi > 75000) & (agi < 100000)]).sum())
    pct11 = round(agi_11 / total, 2)
    agi_12 = ((item[(agi > 100000) & (agi < 200000)] *
               weight[(agi > 100000) & (agi < 200000)]).sum())
    pct12 = round(agi_12 / total, 2)
    agi_13 = ((item[(agi > 200000) & (agi < 500000)] *
               weight[(agi > 200000) & (agi < 500000)]).sum())
    pct13 = round(agi_13 / total, 2)
    agi_14 = ((item[(agi > 500000) & (agi < 1000000)] *
               weight[(agi > 500000) & (agi < 1000000)]).sum())
    pct14 = round(agi_14 / total, 2)
    agi_15 = ((item[(agi > 1000000) & (agi < 1500000)] *
               weight[(agi > 1000000) & (agi < 1500000)]).sum())
    pct15 = round(agi_15 / total, 2)
    agi_16 = ((item[(agi > 1500000) & (agi < 2000000)] *
               weight[(agi > 1500000) & (agi < 2000000)]).sum())
    pct16 = round(agi_16 / total, 2)
    agi_17 = ((item[(agi > 2000000) & (agi < 5000000)] *
               weight[(agi > 2000000) & (agi < 5000000)]).sum())
    pct17 = round(agi_17 / total, 2)
    agi_18 = ((item[(agi > 5000000) & (agi < 10000000)] *
               weight[(agi > 5000000) & (agi < 10000000)]).sum())
    pct18 = round(agi_18 / total, 2)
    agi_19 = (item[agi > 10000000] * weight[agi > 10000000]).sum()
    pct19 = round(agi_19 / total, 2)
    df = [agi_1, agi_2, agi_3, agi_4, agi_5, agi_6, agi_7, agi_8, agi_9,
          agi_10, agi_11, agi_12, agi_13, agi_14, agi_15, agi_16, agi_17,
          agi_18, agi_19]
    pct = [pct1, pct2, pct3, pct4, pct5, pct6, pct7, pct8, pct9, pct10, pct11,
           pct12, pct13, pct14, pct15, pct16, pct17, pct18, pct19]
    index = ['Zero or Negative', '$1-$5K', '$5K-$10K', '$10K-$15K',
             '$15K-$20K', '$20K-$25K', '$25K-$30K', '$30K-$40K',
             '$40K-$50K', '$50K-$75K', '$75K-$100K', '$100K-$200K',
             '$200K-$500K', '$500K-$1M', '$1M-$1.5M', '$1.5M-$2M',
             '$2M-$5M', '$5M-$10M', '$10M and over']
    return df, pct, index

In [8]:
def run_calc(calc, yr):
    calc.advance_to_year(yr)
    calc.calc_all()
    iitax = calc.weighted_total("iitax")
    ptax = calc.weighted_total("payrolltax")
    combined = calc.weighted_total("combined")
    return iitax, ptax, combined

In [9]:
def distplot(var, income_measure='expanded_income', result_type='pct'):
    def getdata(calc, var, income_measure):
        agg, pct, index = distribution(
            calc.array(var), calc.array('s006'), calc.array(income_measure)
        )
        return agg, pct, index
    pltdata = pd.DataFrame()
    agg, pct, index = getdata(puf_calc, var, income_measure)
    if result_type == 'pct':
        pltdata['PUF'] = pct
    else:
        pltdata['PUF'] = [_ * 1e-9 for _ in agg]
    agg, pct, index = getdata(old_cps, var, income_measure)
    if result_type == 'pct':
        pltdata['Old CPS'] = pct
    else:
        pltdata['Old CPS'] = [_ * 1e-9 for _ in agg]
    agg, pct, index = getdata(new_cps, var, income_measure)
    if result_type == 'pct':
        pltdata["New CPS"] = pct
    else:
        pltdata["New CPS"] = [_ * 1e-9 for _ in agg]
    pltdata['index'] = index
    melted = pd.melt(pltdata, id_vars='index')
    if result_type == 'pct':
        y_label = 'Percent of Total'
        y_format="%"
    else:
        y_label = 'Total (billions)'
        y_format = '$.3f'
    plt = alt.Chart(
        melted, width=475
    ).mark_circle(size=50).encode(
        alt.X('index:O', sort=index,
              axis=alt.Axis(title="Expanded Income Bin")),
        alt.Y('value',
              axis=alt.Axis(format=y_format, title=y_label)),
        color='variable'
    )
    return plt

In [10]:
puf_calc = tc.Calculator(
    records=tc.Records(puf), policy=tc.Policy()
)
puf_calc.advance_to_year(START_YR)
puf_calc.calc_all()
old_cps = tc.Calculator(
    records=tc.Records.cps_constructor(), policy=tc.Policy()
)
old_cps.advance_to_year(START_YR)
old_cps.calc_all()
new_cps = tc.Calculator(
    records=tc.Records(data=cps, weights=cps_weights, adjust_ratios=None, start_year=2014),
    policy=tc.Policy()
)
new_cps.advance_to_year(START_YR)
new_cps.calc_all()

# Income and benefit totals and distribution

In [11]:
inc_vars = [
    "c00100", "e00200", "e00900", "e00800", "e00600",
    "e00300", "e00400", "e01100", "e02100", "expanded_income"
]
inc_totals = defaultdict(list)
for var in inc_vars:
    inc_totals[var].append(puf_calc.weighted_total(var))
    inc_totals[var].append(old_cps.weighted_total(var))
    inc_totals[var].append(new_cps.weighted_total(var))
inc_df = pd.DataFrame(inc_totals)
inc_df.index = ["PUF", "Old CPS", "New CPS"]
(inc_df * 1e-9).round(3)

,c00100,e00200,e00900,e00800,e00600,e00300,e00400,e01100,e02100,expanded_income
PUF,12581.194,8396.920,404.321,18.269,374.824,118.251,79.548,4.875,-3.941,14607.136
Old CPS,11545.023,8589.095,491.253,3.528,361.454,127.223,85.845,495.643,189.311,15074.108
New CPS,11090.505,8309.845,447.161,7.380,372.411,100.588,85.262,1034.238,18.424,14885.497


In [12]:
agi_dist = distplot('c00100', 'expanded_income')
agi_dist.title = "Distribution of AGI"
agi_dist

alt.Chart(...)

In [13]:
iitax_dist = distplot('iitax')
iitax_dist.title = "Percentage of Total Income Tax Liability"
iitax_dist

alt.Chart(...)

In [14]:
iitax_agg = distplot('iitax', result_type='sum')
iitax_agg.title = "Total Income Tax Liability"
iitax_agg

alt.Chart(...)

In [15]:
cg_dist = distplot("e01100")
cg_dist.title = "Distribution of Capital Gains"
cg_dist

alt.Chart(...)

In [16]:
cg_agg = distplot('e01100', result_type='sum')
cg_agg.title = "Total Capital Gains"
cg_agg

alt.Chart(...)

In [17]:
exp_inc_dist = distplot("expanded_income")
exp_inc_dist.title = "Distribution of Expanded Income"
exp_inc_dist

alt.Chart(...)

In [18]:
exp_inc_agg = distplot("expanded_income", result_type="sum")
exp_inc_agg.title = "Total Expanded Income"
exp_inc_agg

alt.Chart(...)

In [19]:
wt_dist_plot = distplot("s006")
wt_dist_plot.title = "Weight Distribution"
wt_dist_plot

alt.Chart(...)

In [20]:
wt_agg_plot = distplot("s006", result_type="sum")
wt_agg_plot.title = "Total Weight"
wt_agg_plot

alt.Chart(...)

# Aggregate Totals

In [21]:
aggs = defaultdict(list)
agg_plot_data = defaultdict(list)
for yr in range(START_YR, END_YR + 1):
    puf_aggs = run_calc(puf_calc, yr)
    old_cps_aggs = run_calc(old_cps, yr)
    new_cps_aggs = run_calc(new_cps, yr)
    aggs["puf_iitax"].append(puf_aggs[0])
    aggs["puf_ptax"].append(puf_aggs[1])
    aggs["puf_combined"].append(puf_aggs[2])
    aggs["old_cps_iitax"].append(old_cps_aggs[0])
    aggs["old_cps_ptax"].append(old_cps_aggs[1])
    aggs["old_cps_combined"].append(old_cps_aggs[2])
    aggs["new_cps_iitax"].append(new_cps_aggs[0])
    aggs["new_cps_ptax"].append(new_cps_aggs[1])
    aggs["new_cps_combined"].append(new_cps_aggs[2])
    aggs['year'].append(yr)

In [22]:
aggs_df = pd.DataFrame(aggs)
aggs_df * 1e-9

,puf_iitax,puf_ptax,puf_combined,old_cps_iitax,old_cps_ptax,old_cps_combined,new_cps_iitax,new_cps_ptax,new_cps_combined,year
0,1654.099056,1209.406122,2863.505179,1352.728022,1262.668270,2615.396292,1101.042378,1218.419321,2319.461698,0.000002
1,1722.262885,1266.211839,2988.474724,1450.552952,1323.690983,2774.243935,1158.164460,1275.566438,2433.730898,0.000002
2,1785.755433,1319.632227,3105.387660,1536.764722,1381.122725,2917.887447,1210.030288,1329.179130,2539.209418,0.000002
3,1850.145542,1371.545157,3221.690699,1616.172176,1436.209855,3052.382031,1259.293447,1380.527873,2639.821320,0.000002
4,1926.123660,1425.728785,3351.852444,1697.954644,1493.355012,3191.309656,1315.016256,1434.500648,2749.516905,0.000002
5,2017.056379,1483.697940,3500.754319,1783.756362,1554.302861,3338.059223,1379.553683,1491.774559,2871.328242,0.000002
6,2112.754399,1543.481728,3656.236127,1865.411599,1617.198774,3482.610373,1446.990749,1551.358184,2998.348933,0.000002
7,2414.812837,1604.064981,4018.877818,2151.694657,1680.622537,3832.317194,1703.685918,1611.909878,3315.595796,0.000002
8,2530.872952,1667.331326,4198.204277,2253.926199,1747.077765,4001.003964,1781.820797,1675.702244,3457.523041,0.000002
9,2653.139048,1733.859197,4386.998245,2361.886643,1816.988452,4178.875095,1865.095226,1743.262080,3608.357305,0.000002


In [23]:
iitax_plot_data = pd.melt(aggs_df, id_vars='year', value_vars=['puf_iitax', 'old_cps_iitax', 'new_cps_iitax'])
ptax_plot_data = pd.melt(aggs_df, id_vars='year', value_vars=['puf_ptax', 'old_cps_ptax', 'new_cps_ptax'])
combined_plot_data = pd.melt(aggs_df, id_vars='year', value_vars=['puf_combined', 'old_cps_combined', 'new_cps_combined'])

In [24]:
alt.Chart(combined_plot_data, title="Combined Tax", width=500).mark_line().encode(
    alt.X('year:O'),
    alt.Y('value'),
    color='variable'
)

alt.Chart(...)

In [25]:
alt.Chart(iitax_plot_data, title="Income Tax", width=500).mark_line().encode(
    alt.X('year:O'),
    alt.Y('value'),
    color='variable'
)

alt.Chart(...)

In [26]:
alt.Chart(ptax_plot_data, title="Payroll Tax", width=500).mark_line().encode(
    alt.X('year:O'),
    alt.Y('value'),
    color='variable'
)

alt.Chart(...)